# General graph functions (different ways of describing a graph)

In [7]:
A = [(0,2),(0,1), (2,0), (1,0)] # A graph defined by its edges with vertices being integers 


def compter_sommets(arretes): # Counts the number of vertices
    return max([max(el) for el in arretes]) + 1


def ens_sommet(arretes): # Function that gives the set of all the vertices
    ens = []
    for arr in arretes:
        for i in range(2):
            ens.append(arr[i])
    return set(ens)

print(f"The set of vertices of graph A is {ens_sommet(A)}")


def arretes2voisins(arretes): # Gives the dictionary of all the vertices with its neighboors in the graph
    ens = ens_sommet(arretes)
    voisins = {edge : [] for edge in ens}
    for edge in ens:
        for neighboor in ens:
            if (edge, neighboor) in arretes :
                voisins[edge].append(neighboor)
    return voisins

print(f"The neighboors of each vertices are: {arretes2voisins(A)}")


def voisins2arretes(voisins): # Opposite function of arretes2voisins - gives the list of the edges (vetice 1, vertice 2)
    arretes = []
    for edge in voisins:
        for v in voisins[edge]:
            arretes.append((edge, v))
    return arretes
                
print(f"The different edges are: {voisins2arretes(arretes2voisins(A))}")


def distance(origine,voisins, dist = False): # Provides a dictionary with the vertices and their distance to a particular vertice
    if not dist:
        dist = {edge : -1 for edge in voisins.keys()} # Initial value of dist
    d = 0
    while len(origine) !=0 :
        for sommet in origine:
            dist[sommet] = d # We can pass multiple times on the same vertice (no worries)
        d += 1
        origine = [s for sommet in origine for s in voisins[sommet] if dist[s] == -1] # Duplicates doesn't matter
    return dist


def num_composantes(voisins): # Counts the connected components
    dist = {edge : -1 for edge in voisins}
    num_comp = 0
    for edge in dist:
        if dist[edge] == -1:
            num_comp += 1
            dist = distance([edge], voisins, dist)
    return num_comp



print("\nTesting functions for a different graph")
arr = [(0,1), (1,2), (2,1), (3,0), (0,3)]
num = num_composantes(arretes2voisins(arr))

pluriel = 's' if num > 1 else ''
print(f'The graph has {num} connected component{pluriel}')

vois = arretes2voisins(arr)
print(f'The distances of each vertice to the vertice 3 are {distance([3], vois)}')

The set of vertices of graph A is {0, 1, 2}
The neighboors of each vertices are: {0: [1, 2], 1: [0], 2: [0]}
The different edges are: [(0, 1), (0, 2), (1, 0), (2, 0)]

Testing functions for a different graph
The graph has 1 connected component
The distances of each vertice to the vertice 3 are {0: 1, 1: 2, 2: 3, 3: 0}


# Jeu du Taquin / Game of 15

In [2]:
# Very simple code to obtain the two possible next positions in the Taquin 2*2
# Not very efficient or useful just to understand the game
Total = []
tab = ['a','b','c','X']

for i in range (4):
    if tab[i]== 'X':
        y = tab[:]
        tab[(i+1)%4],tab[i] = tab[i], tab[(i+1)%4]
        if tab not in Total:
            Total.append(tab)
        y[(i-1)%4],y[i] = y[i],y[(i-1)%4]
        if y not in Total:
            Total.append(y)
            
print(Total)

[['X', 'b', 'c', 'a'], ['a', 'b', 'X', 'c']]


In [19]:
# Direct generalisation to the Taquin-like game played around a circle 

from itertools import *

def voisins(position): # Function giving the neighboors of each state of the Circle Taquin game
    taille = len(position)
    V = []
    position = list(position)
    for i in range (taille):
        if position[i]== 0:
            pos = position[:]
            position[(i+1)% taille],position[i] = position[i], position[(i+1)% taille]
            if position not in V:
                V.append(tuple(position))
            pos[(i-1)% taille], pos[i] = pos[i],pos[(i-1)% taille]
            if pos not in V:
                V.append(tuple(pos))
    return V


taille = 7      # Number of possible spots around the necklaces so there are 6 beads, because there is an empty spot
vertices = []   # List of all the vertices
graph = {}      # Complete graph of the Game

for sigma in permutations(range(taille)):  # Gives the complete graph of the Game through a dictionary where every vertice is associated with a list of its neighboors
    vertices.append(sigma)
    graph[sigma] = voisins(sigma)


def maxdist(dicodistance): # Gives the vertice that is the furthest away from a fixed origin
    maxi = -1

    for vertice in dicodistance:
        if dicodistance[vertice] > maxi:
            maxi = dicodistance[vertice]
            verticemax = vertice
    return verticemax, dicodistance[verticemax]

dicodistance = distance([tuple(range(taille-1,-1,-1))], graph) # Gives every vertice's distance from the starting point in the graph of the Circle Taquin game


composante_connexe = []
l = []

for vertice in dicodistance:
    if dicodistance[vertice] >= 0: # Checks if the vertice is in the original connected component
        l.append((vertice.index(0), vertice.index(1))) # Gives the position of the 0 and 1 (which is sufficient to determine the current state) for every vertice in a same connected component
        composante_connexe.append(vertice)


print(f'There are {len(composante_connexe)} elements in the original connected component')
print(len(l) == len(set(l))) # Shows that the positions of the 0 and 1 determine the current state

print(f"\n{composante_connexe}\n") # Provides the whole list of the vertices in the original connected component - SHOWS THE INVARIANT OF THE CYCLIC ORDER


print(maxdist(dicodistance))
print(f'{tuple(range(taille-1,-1,-1))} is at a distance of {dicodistance[tuple(range(taille-1,-1,-1))]} from the origin\n')


num = num_composantes(graph)
pluriel = 's' if num > 1 else ''
print(f'The graph has {num} connected component{pluriel}')

There are 42 elements in the original connected component
True

[(0, 1, 6, 5, 4, 3, 2), (0, 2, 1, 6, 5, 4, 3), (0, 3, 2, 1, 6, 5, 4), (0, 4, 3, 2, 1, 6, 5), (0, 5, 4, 3, 2, 1, 6), (0, 6, 5, 4, 3, 2, 1), (1, 0, 6, 5, 4, 3, 2), (1, 6, 0, 5, 4, 3, 2), (1, 6, 5, 0, 4, 3, 2), (1, 6, 5, 4, 0, 3, 2), (1, 6, 5, 4, 3, 0, 2), (1, 6, 5, 4, 3, 2, 0), (2, 0, 1, 6, 5, 4, 3), (2, 1, 0, 6, 5, 4, 3), (2, 1, 6, 0, 5, 4, 3), (2, 1, 6, 5, 0, 4, 3), (2, 1, 6, 5, 4, 0, 3), (2, 1, 6, 5, 4, 3, 0), (3, 0, 2, 1, 6, 5, 4), (3, 2, 0, 1, 6, 5, 4), (3, 2, 1, 0, 6, 5, 4), (3, 2, 1, 6, 0, 5, 4), (3, 2, 1, 6, 5, 0, 4), (3, 2, 1, 6, 5, 4, 0), (4, 0, 3, 2, 1, 6, 5), (4, 3, 0, 2, 1, 6, 5), (4, 3, 2, 0, 1, 6, 5), (4, 3, 2, 1, 0, 6, 5), (4, 3, 2, 1, 6, 0, 5), (4, 3, 2, 1, 6, 5, 0), (5, 0, 4, 3, 2, 1, 6), (5, 4, 0, 3, 2, 1, 6), (5, 4, 3, 0, 2, 1, 6), (5, 4, 3, 2, 0, 1, 6), (5, 4, 3, 2, 1, 0, 6), (5, 4, 3, 2, 1, 6, 0), (6, 0, 5, 4, 3, 2, 1), (6, 5, 0, 4, 3, 2, 1), (6, 5, 4, 0, 3, 2, 1), (6, 5, 4, 3, 0, 2, 1), (6, 5, 4, 3, 2,

# Rubik's Cube 2x2

In [46]:
rubikcube = {(0,0,0):['O','W','G'],(0,0,1):['O','W','B'],(1,0,1):['R','W','B'],(1,0,0):['R','W','G'],(1,1,0):['R','Y','G'],(1,1,1):['R','Y','B'],(0,1,1):['O','Y','B'],(0,1,0):['O','Y','G']}
# Describe the cube using (x, y, z) coordinates, with the colors representing the labels on the planes perpendicular to each axis
# Inital state of the cube


def next(l, base = 2, retenue = 1): # Function to simply take the "next" element
    l = list(l)
    for k in range(len(l)):
        s = l[k] + retenue
        l[k] = s % base
        retenue = s // base
    return tuple(l)


def repimut(cube): # Function that concatenates the description of the cube - easier to then manipulate for the rotations...
    rep = ''
    point = (0,0,0)
    for _ in range(8):
        rep += ''.join(cube[point])
        point = next(point)
    return rep
# print(repimut(rubikcube))

def tumiper(rep): # Inverse function of repimut
    cube = {}
    point = (0,0,0)
    for k in range(8):
        cube[point] = list(rep[3*k:3*k + 3])
        point = next(point)
    return cube
# print(tumiper(repimut(rubikcube)))
# print(tumiper(repimut(rubikcube))==rubikcube)

def rotation(rubikcube, axe): # Defines the state of the cube after a rotation
    sommets = {'X':[(1,0,0),(1,0,1),(1,1,1),(1,1,0)],'Y':[(0,1,0),(1,1,0),(1,1,1),(0,1,1)],'Z':[(0,0,1),(0,1,1),(1,1,1),(1,0,1)]}
    Changement = {'X':[0,2,1],'Y':[2,1,0],'Z':[1,0,2]}
    cube_final = rubikcube.copy()
    for i in range(4):
        cube_final[sommets[axe][i]] = [rubikcube[sommets[axe][(i-1)%4]][Changement[axe][j]] for j in range(3)]
    return cube_final

# print(repimut(rotation(rubikcube,"X")))


"""
def graph(depart, voisins): # First version to compute the graph of a connected component of the cube (IMPROVED VERSION BELOW)
    V = {}
    dist = {}
    d = 0
    cpt = 0
    while len(depart) != 0:
        dep = []
        for vertice in depart:
            if vertice not in V.keys():
                cpt += 1
                if cpt % 100000 == 0:
                    print(cpt, d, vertice)
                vois = voisins(vertice)
                V[vertice] = vois
                if vertice not in dist :
                    dist[vertice] = d
                dep += vois

        depart = dep
        d += 1
    return V, dist, d
"""
# Generators / Different Elementary moves
def voisins(cube): # Quarter turns in one direction (Returns (Move/Rotation, Initial Position, Final Position))
    v = []
    for axe in 'XYZ':
        cube_copie = cube
        cube_copie = repimut(rotation(tumiper(cube_copie),axe))
        v.append((axe + str(1), cube, cube_copie))
    return v

def voisins2(cube): # Quarter turns in both directions (Returns (Move/Rotation, Initial Position, Final Position))
    v = []
    for axe in 'XYZ':
        cube_copie = cube
        for i in range(3):
            cube_copie = repimut(rotation(tumiper(cube_copie),axe))
            if i != 1:
                v.append((axe + str(i+1), cube, cube_copie))
    return v


def voisins3(cube): # Quarter turns in both directions, and half turns (Returns (Move/Rotation, Initial Position, Final Position))
    v = []
    for axe in 'XYZ':
        cube_copie = cube
        for i in range(3):
            cube_copie = repimut(rotation(tumiper(cube_copie),axe))
            v.append((axe + str(i+1), cube, cube_copie))
    return v

# print(voisins3(repimut(rubikcube)))


def graph_etiquette(depart, voisins): # Breadth-First Search algorithm: computes the complete graph of a connected component of the cube, offering the path to the furthest state away
    V = {}
    chemin = {edge : '' for edge in depart}
    cpt = 0

    vois = []
    for edge in depart:
        vois += voisins(edge)
    
    while len(vois) != 0:
        vois_next = []
        for (etiquette, origine, arrivee) in vois:
            if arrivee not in V.keys():
                cpt += 1
                if cpt % 100000 == 0:
                    print(cpt, arrivee)
                v = voisins(arrivee)
                V[arrivee] = [x[2] for x in v]
                chemin[arrivee] = chemin[origine] + etiquette
                vois_next += v

        vois = vois_next
    return V, chemin


V, chemin = graph_etiquette([repimut(rubikcube)], voisins2)


print(len(V)) # Gives the number of states in the original connected component

#print(V) # DO NOT EXECUTE !!! Prints all the state in the original connected component 

100000 OWGRWGYBRWOBRGYRWBOYBYGO
200000 OWGWBORYBGWRGYRBRWBOYOYG
300000 OWGYRGWRBBYRBYOGYOWRGBOW
400000 OWGGRWBOWRBYOYGWBRGRYYOB
500000 OWGRBWOBYGWRYGORBYBWOGYR
600000 OWGWBORBWRGWRGYYOGRBYBYO
700000 OWGYGOYBRRYGWBOWRGBRWBYO
800000 OWGWGRYBRRYGWRBOBWOYBGOY
900000 OWGWBOWRBYGRRYBOGYRGWBYO
1000000 OWGWGRBYOOBWRGYYOGRBYRBW
1100000 OWGWBOWGRRBYRBWRYGYOGOBY
1200000 OWGRGYOYGWRGBRYYBOBWOWRB
1300000 OWGYOBRYBBWOYRGOGYRGWRBW
1400000 OWGOWBGOYWRGBWRYRBOYBGYR
1500000 OWGGOYWRBWOBYRGGWROYBRYB
1600000 OWGWBOWGRYGRBYOBYRWBROYG
1700000 OWGWRBWGRGYOBOWGRYBOYRYB
1800000 OWGGRWRYBYGRBWRBWOGYOOBY
1900000 OWGGOYYBRWOBGYROYBWRGRBW
2000000 OWGBOWGRWOYBRBWRBYYGROYG
2100000 OWGRGYYBRBOYRWGOGYBWOWRB
2200000 OWGOYGBWRBOYBRYYGRWRGBOW
2300000 OWGYBRYOBWBRRGYRGWOGYWBO
2400000 OWGGRWWBOYGRRYBYBOWBROYG
2500000 OWGRGYBYOOBWRBWWRGRBYYGO
2600000 OWGYGOBRYYBORBWWRGGRYBOW
2700000 OWGYGOWBORGWBYOBYRGRYRBW
2800000 OWGRGYBYOOBWGOYRBYWRGWRB
2900000 OWGGYRWBOBOYYBRYOGBRWRWG
3000000 OWGOWBYBRRGWYOBYGROGYBWR
3100000 OWGOWBRBWGW

# Final Results

In [48]:
m = max([len(chem)//2 for chem in chemin.values()]) # Computes the diameter, maximum distance from the origin
print(m)

loin = [edge for edge in chemin if len(chemin[edge])//2==m] # The set of all the cubes farthest away from the origin
print(len(loin)) # Number of states the furthest away
print(tumiper(loin[0])) # Example of a cube that is the furthest distance away 
print(chemin[loin[0]]) # Moves to access one of the furthest cube

14
276
{(0, 0, 0): ['O', 'W', 'G'], (1, 0, 0): ['O', 'B', 'Y'], (0, 1, 0): ['B', 'O', 'W'], (1, 1, 0): ['G', 'Y', 'O'], (0, 0, 1): ['G', 'R', 'W'], (1, 0, 1): ['Y', 'R', 'B'], (0, 1, 1): ['R', 'W', 'B'], (1, 1, 1): ['Y', 'R', 'G']}
X1X1Y1X1X1Y1X1Y3Z1Y1Y1Z1Y3Z3


# Extra - 2 Ways to Visualize the Rubik's Cube 2x2

In [3]:
# Textual flattened representation of the cube

#Rubik's Cube sides
#           (1,Z)       
# (0,X)     (0,Y)       (1,X)       (1,Y)
#           (0,Z)       


# Fixed small cube : (0,0,0)

rubikcube = {(0,0,0):['O','W','G'],(0,0,1):['O','W','B'],(1,0,1):['R','W','B'],(1,0,0):['R','W','G'],(1,1,0):['R','Y','G'],(1,1,1):['R','Y','B'],(0,1,1):['O','Y','B'],(0,1,0):['O','Y','G']}
rubikcube1 = {(0, 0, 0): ['O', 'W', 'G'], (1, 0, 0): ['O', 'B', 'Y'], (0, 1, 0): ['B', 'O', 'W'], (1, 1, 0): ['G', 'Y', 'O'], (0, 0, 1): ['G', 'R', 'W'], (1, 0, 1): ['Y', 'R', 'B'], (0, 1, 1): ['R', 'W', 'B'], (1, 1, 1): ['Y', 'R', 'G']}


def next(l, base = 2, retenue = 1): # Function to simply take the "next" element
    l = list(l)
    for k in range(len(l)):
        s = l[k] + retenue
        l[k] = s % base
        retenue = s // base
    return tuple(l)


def repimut(cube): # Function that concatenates the description of the cube - easier to then manipulate for the rotations...
    rep = ''
    point = (0,0,0)
    for _ in range(8):
        rep += ''.join(cube[point])
        point = next(point)
    return rep

def final(rubikcube):
    Solutions = repimut(rubikcube)
    
    print('-     -', end="        ")
    print(f"{Solutions[20]}     {Solutions[23]}", end="        ")
    print('-     -', end="        ")
    print('-     -')

    print('-     -', end="        ")
    print(f"{Solutions[14]}     {Solutions[17]}", end="        ")
    print('-     -', end="        ")
    print('-     -')

    print('\n')
    print(f"{Solutions[18]}     {Solutions[12]}", end="        ")
    print(f"{Solutions[13]}     {Solutions[16]}", end="        ")
    print(f"{Solutions[15]}     {Solutions[21]}", end="        ")
    print(f"{Solutions[22]}     {Solutions[19]}")

    print(f"{Solutions[12]}     {Solutions[0]}", end="        ")
    print(f"{Solutions[1]}     {Solutions[4]}", end="        ")
    print(f"{Solutions[3]}     {Solutions[9]}", end="        ")
    print(f"{Solutions[10]}     {Solutions[7]}")
    print('\n')

    print('-     -', end="        ")
    print(f"{Solutions[2]}     {Solutions[5]}", end="        ")
    print('-     -', end="        ")
    print('-     -')

    print('-     -', end="        ")
    print(f"{Solutions[8]}     {Solutions[11]}", end="        ")
    print('-     -', end="        ")
    print('-     -')

final(rubikcube1)


-     -        B     G        -     -        -     -
-     -        W     B        -     -        -     -


R     G        R     R        Y     Y        R     W
G     O        W     B        O     G        Y     O


-     -        G     Y        -     -        -     -
-     -        W     O        -     -        -     -


In [5]:
# Graphical flattened representation of the cube

import turtle as t
import time

def next(l, base = 2, retenue = 1): # Function to simply take the "next" element
    l = list(l)
    for k in range(len(l)):
        s = l[k] + retenue
        l[k] = s % base
        retenue = s // base
    return tuple(l)

def repimut(cube): # Function that concatenates the description of the cube - easier to then manipulate for the rotations...
    rep = ''
    point = (0,0,0)
    for _ in range(8):
        rep += ''.join(cube[point])
        point = next(point)
    return rep


rubikcube = {(0,0,0):['O','W','G'],(0,0,1):['O','W','B'],(1,0,1):['R','W','B'],(1,0,0):['R','W','G'],(1,1,0):['R','Y','G'],(1,1,1):['R','Y','B'],(0,1,1):['O','Y','B'],(0,1,0):['O','Y','G']}


colors = {'O':'Orange','W':'White','B':'Blue','R':'Red','G':'Green','Y':'Yellow'}
Solutions = repimut(rubikcube)

couleur_en_ordre = {(-80,120):colors[Solutions[20]], (-40,120):colors[Solutions[23]], (-80,80):colors[Solutions[14]], (-40,80):colors[Solutions[17]], (-160,40):colors[Solutions[18]], (-120,40):colors[Solutions[12]], (-80,40):colors[Solutions[13]], (-40,40):colors[Solutions[16]], (0,40):colors[Solutions[15]], (40,40):colors[Solutions[21]], (80,40):colors[Solutions[22]], (120,40):colors[Solutions[19]], (-160,0):colors[Solutions[12]], (-120,0):colors[Solutions[0]], (-80,0):colors[Solutions[1]], (-40,0):colors[Solutions[4]], (0,0):colors[Solutions[3]], (40,0):colors[Solutions[9]], (80,0):colors[Solutions[10]], (120,0):colors[Solutions[7]], (-80,-40):colors[Solutions[2]], (-40,-40):colors[Solutions[5]], (-80,-80):colors[Solutions[8]], (-40,-80):colors[Solutions[11]]}



t.speed(500)
for el in couleur_en_ordre:
    t.penup()
    t.goto(el)
    t.pendown()


    t.fillcolor(couleur_en_ordre[el])
    t.begin_fill()
    for _ in range(4):
        t.forward(40)
        t.right(90)
    t.end_fill()

time.sleep(4)
t.bye()